In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Feature Engineering para el Modelo Parte 1

## Cargar Datos

In [12]:
completo_df = pd.read_csv('../data/raw/df_clean_completo.csv')
print("Datos cargados exitosamente.")
completo_df

Datos cargados exitosamente.


,COD_PERSONA,COD_ALUMNO,SEXO,PER_INGRESO,ESTADO_CIVIL,TIPO_COLEGIO,PTJE_INGRESO,ESTADO,SEM_CURSADOS,CANT_RESERVAS,...,CRED_GRADUACION,BECA_VIGENTE,FECHA_NACIMIENTO,POBREZA_RES,POBREZA_PRO,TIPO_CICLO,CODIGO,SEM,NIVEL_CURSO,FAMILIA
0,15788,15798,M,2011-01,S,Privada Particular,120.0,Retirado,2,0,...,220,0,1993-12-28,0.715245,5.430922,Regular,CS1D1,1,1,CS
1,14933,14938,M,2011-01,S,Privada Particular,135.0,Separado,6,1,...,220,0,1993-07-22,0.345310,0.715245,Regular,FG101,1,1,FG
2,14933,14938,M,2011-01,S,Privada Particular,135.0,Separado,6,1,...,220,0,1993-07-22,0.345310,0.715245,Regular,CS1D1,1,1,CS
3,14933,14938,M,2011-01,S,Privada Particular,135.0,Separado,6,1,...,220,0,1993-07-22,0.345310,0.715245,Regular,MA100,1,1,MA
4,22813,16125,M,2011-02,S,Pública otro Sector Público,105.0,Retirado,6,0,...,220,0,1993-09-11,1.433882,0.715245,Regular,MA100,1,1,MA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26997,33223,32540,M,2017-01,S,Privada Particular,113.0,Retirado,7,1,...,219,0,1991-02-25,0.345310,0.715245,Regular,CS1D2,2,2,CS
26998,33223,32540,M,2017-01,S,Privada Particular,113.0,Retirado,7,1,...,219,0,1991-02-25,0.345310,0.715245,Regular,CS100,2,2,CS
26999,33223,32540,M,2017-01,S,Privada Particular,113.0,Retirado,7,1,...,219,0,1991-02-25,0.345310,0.715245,Regular,CS112,2,2,CS
27000,33223,32540,M,2017-01,S,Privada Particular,113.0,Retirado,7,1,...,219,0,1991-02-25,0.345310,0.715245,Regular,MA100,1,1,MA


## Crear Cluster Cursos
Cargar JSON de descripcion de cursos
str -> tf-idf -> SVD(20 components) -> Kmeans

In [13]:
import json
import pandas as pd

# Cargar el JSON
with open('../data/raw/cursos.json', 'r', encoding='utf-8') as archivo:
    cursos_data = json.load(archivo)
# Dataframe
cursos_nlp_df = pd.DataFrame(cursos_data)
cursos_nlp_df['DESCRIPCION_TEXTO'] = cursos_nlp_df['DESCRIPCION'].apply(lambda x: ' '.join(x))
print(cursos_nlp_df[['CODIGO', 'DESCRIPCION_TEXTO']].head())

  CODIGO                                  DESCRIPCION_TEXTO
0  CS111  programming game development algorithms logic ...
1  CS1D1  mathematics logic sets relations functions pro...
2  FG101  communication writing reading speaking listeni...
3  FG102  study techniques learning methods time managem...
4  FG103  university life academic adaptation campus res...


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=500, stop_words='english')

features_tfidf = vectorizer.fit_transform(cursos_nlp_df['DESCRIPCION_TEXTO'])

print("\nForma de la matriz TF-IDF (cursos, features):")
print(features_tfidf.shape)


Forma de la matriz TF-IDF (cursos, features):
(71, 500)


In [15]:
from sklearn.decomposition import TruncatedSVD

n_componentes = 20
svd = TruncatedSVD(n_components=n_componentes, random_state=42)
features_reducidos = svd.fit_transform(features_tfidf)

print(f"\nForma de la matriz reducida (cursos, componentes):")
print(features_reducidos.shape)


Forma de la matriz reducida (cursos, componentes):
(71, 20)


In [16]:
from sklearn.cluster import KMeans

# Definimos cuántos clusters queremos encontrar (ej. 8)
# (Puedes usar el "Método del Codo" para encontrar el 'k' óptimo)
n_clusters = 8
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)

# Asignamos cada curso a un cluster
clusters = kmeans.fit_predict(features_reducidos)

# ¡Listo! Añadimos el cluster como un nuevo feature a nuestro DataFrame
cursos_nlp_df['CLUSTER_CURSO'] = clusters

print("\nCursos con su cluster asignado:")
print(cursos_nlp_df[['CODIGO', 'CURSO', 'CLUSTER_CURSO']].head())


Cursos con su cluster asignado:
  CODIGO                                 CURSO  CLUSTER_CURSO
0  CS111          Programación de Video Juegos              7
1  CS1D1               Estructuras Discretas I              4
2  FG101                          Comunicación              0
3  FG102               Metodología del Estudio              6
4  FG103  Introducción a la Vida Universitaria              3


In [17]:
cursos_nlp_df[['CODIGO', 'CURSO', 'CLUSTER_CURSO']]

,CODIGO,CURSO,CLUSTER_CURSO
0,CS111,Programación de Video Juegos,7
1,CS1D1,Estructuras Discretas I,4
2,FG101,Comunicación,0
3,FG102,Metodología del Estudio,6
4,FG103,Introducción a la Vida Universitaria,3
...,...,...,...
66,CS404,Proyecto Final de Carrera III,6
67,ET301,Formación de Empresas de Base Tecnológica II,2
68,FG211,Ética Profesional,3
69,FG220,Análisis de la Realidad Peruana,3


In [18]:
for cluster in cursos_nlp_df['CLUSTER_CURSO'].unique():
    print(cursos_nlp_df['CODIGO'][cursos_nlp_df['CLUSTER_CURSO'] == cluster].unique())

['CS111' 'CS100' 'CS112' 'CS221' 'CS231' 'CS251' 'CS261' 'CS341' 'CS2H1'
 'CS342' 'CS361']
['CS1D1' 'CS1D2' 'CS113' 'CS210' 'CS212' 'CS311' 'CS312']
['FG101' 'FG203' 'ID101']
['FG102' 'CS281' 'ET101' 'CS402' 'CS381' 'CS403' 'CS362' 'CS404']
['FG103' 'FG104' 'FG105' 'FG111' 'FG107' 'FG201' 'FG202' 'CS211' 'FG204'
 'FG106' 'FG210' 'CS201' 'FG205' 'FG221' 'FG301' 'FG211' 'FG220']
['MA100' 'MA101' 'CS1D3' 'MA102' 'MA201' 'MA203' 'CB111' 'MA306' 'MA307']
['CS271' 'CS272' 'CS331' 'FG350' 'CS351' 'CB309' 'CS370' 'ET201' 'CS393'
 'ET301']
['CS291' 'CS292' 'CS391' 'CS3P1' 'CS392' 'CS3P2']


## Unir features

In [19]:
cursos_clusters = cursos_nlp_df[['CODIGO', 'CLUSTER_CURSO']]

# 'left merge'
completo_df = pd.merge(
    completo_df,                 # DataFrame principal (izquierda)
    cursos_clusters,             # DataFrame con los clusters (derecha)
    left_on='COD_CURSO',         # Llave en el DataFrame principal
    right_on='CODIGO',           # Llave equivalente en el DataFrame de clusters
    how='left'                   # Mantiene todas las filas de 'completo_df'
)

In [20]:
import os

os.makedirs('../data/processed', exist_ok=True)

file_path = '../data/processed/completo_df_v1.parquet'

completo_df.to_parquet(file_path, index=False, engine='fastparquet')

print(f"DataFrame guardado exitosamente como 'version 1' en: {file_path}")

DataFrame guardado exitosamente como 'version 1' en: ../data/processed/completo_df_v1.parquet
